## Imports

In [397]:
import os
import openai
import nest_asyncio
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import chromadb
from chromadb import Client
from sentence_transformers import SentenceTransformer
import re

## Initialisierung

In [398]:
# Laden der Umgebungsvariablen
load_dotenv()

# OpenAI Client initialisieren
openAIclient = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

# ChromaDB Client initialisieren
chromaDBclient = Client()
collection_name = "nutrition_documents"

# SentenceTransformer-Modell initialisieren
model = SentenceTransformer('all-MiniLM-L6-v2')

# Event-Loop anpassen
nest_asyncio.apply()

## Überprüfen und Löschen der Sammlung


In [399]:
# Alle vorhandenen Sammlungen auflisten
collections = chromaDBclient.list_collections()

# Namen der Sammlungen extrahieren
collection_names = [coll.name for coll in collections]

# Wenn die Sammlung bereits existiert, wird sie gelöscht
if collection_name in collection_names:
    chromaDBclient.delete_collection(collection_name)

# Eine neue Sammlung erstellen
collection = chromaDBclient.create_collection(name=collection_name)

## Dokumente in Abschnitte aufteilen

In [400]:
def split_documents(text):
      # Text in Absätze aufteilen
      paragraphs = text.split("\n")
      return paragraphs

## Funktion zum Speichern von Dokumenten mit Metadaten

In [414]:
def store_documents_with_metadata(documents, source_file):
   """
   Speichert Dokumente mit Abschnitten, Embeddings und zugehörigen Metadaten in ChromaDB.
   :param documents: Liste von Dokumentobjekten.
   :param source_file: Der Name der Datei, aus der das Dokument stammt.
   """
   for idx, document in enumerate(documents):
      content = document.text
      
      # Eindeutige Dokument-ID basierend auf Dateiname und Index
      doc_id = f"{source_file}_page_{idx}"

      # Abschnittsweise Verarbeitung des Inhalts
      sections = content.split("Kapitel")  # Aufteilung nach Absätzen
      #sections = split_document(content)
      print(f"\nDokument {source_file}, Abschnittsanzahl: {len(sections)}")  # Debug-Ausgabe
      
      for section_idx, section in enumerate(sections):
         if section.strip():  # Leere Abschnitte überspringen
               embedding = model.encode(section)  # Embedding erstellen
               print(f"Speichere Abschnitt {section_idx} für {doc_id}")  # Debug-Ausgabe
               
               # Hinzufügen des Abschnitts zu ChromaDB
               collection.add(
                  documents=[section],
                  metadatas=[{"section_id": section_idx, "page_id": doc_id, "source_file": source_file}],
                  ids=[f"{doc_id}_section_{section_idx}"],
                  embeddings=[embedding]
               )

   # Bestätigungsausgabe
   print(f"\nDokumente aus {source_file} mit Embeddings in ChromaDB gespeichert.")


## Funktion zur Verarbeitung aller Dateien in einem Ordner

In [415]:
def process_folder_with_metadata(folder_path: str):
   
   # Überprüfen, ob der Ordner existiert
   if not os.path.exists(folder_path):
      print(f"Ordner '{folder_path}' existiert nicht.")
      return

   # Alle Dateien im Ordner durchlaufen
   for file_name in os.listdir(folder_path):
      file_path = os.path.join(folder_path, file_name)

      # Nur Dateien (keine Unterordner) verarbeiten
      if os.path.isfile(file_path):
         print("\n---------------------------------------------------------")
         print(f"Lade Datei: {file_name}")
         print("---------------------------------------------------------")
         try:
            # Dokumente aus der Datei laden
            reader = SimpleDirectoryReader(input_files=[file_path])
            documents = reader.load_data()
            if documents:
               store_documents_with_metadata(documents, file_name)
               print(f"Dokument {file_name} erfolgreich verarbeitet.")
            else:
               print(f"Keine Inhalte aus der Datei {file_name} extrahiert.")
         except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei '{file_name}': {e}")

## Ordnerpfad definieren und Funktion aufrufen

In [416]:
# Ordnerpfad zum Verarbeiten der Dateien angeben
folder_path = "./data"

# Verarbeitungsfunktion für den Ordner aufrufen
process_folder_with_metadata(folder_path)

# Abschließende Bestätigungsausgabe
print("Alle Dokumente aus dem Ordner verarbeitet und gespeichert.")

Add of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_0
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_0
Add of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_1
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_1
Add of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_2
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_2
Add of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_3
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_3



---------------------------------------------------------
Lade Datei: fake_nutrition-trends.pdf
---------------------------------------------------------

Dokument fake_nutrition-trends.pdf, Abschnittsanzahl: 5
Speichere Abschnitt 0 für fake_nutrition-trends.pdf_page_0
Speichere Abschnitt 1 für fake_nutrition-trends.pdf_page_0
Speichere Abschnitt 2 für fake_nutrition-trends.pdf_page_0
Speichere Abschnitt 3 für fake_nutrition-trends.pdf_page_0


Add of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_4
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_0_section_4
Add of existing embedding ID: fake_nutrition-trends.pdf_page_1_section_0
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_1_section_0
Add of existing embedding ID: fake_nutrition-trends.pdf_page_1_section_1
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_1_section_1
Add of existing embedding ID: fake_nutrition-trends.pdf_page_1_section_2
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_1_section_2
Add of existing embedding ID: fake_nutrition-trends.pdf_page_2_section_0
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_2_section_0
Add of existing embedding ID: fake_nutrition-trends.pdf_page_2_section_1
Insert of existing embedding ID: fake_nutrition-trends.pdf_page_2_section_1
Add of existing embedding ID: fakte_protein.pdf_page_0_section_0
Insert of existing embedding ID: fakte_pr

Speichere Abschnitt 4 für fake_nutrition-trends.pdf_page_0

Dokument fake_nutrition-trends.pdf, Abschnittsanzahl: 3
Speichere Abschnitt 0 für fake_nutrition-trends.pdf_page_1
Speichere Abschnitt 1 für fake_nutrition-trends.pdf_page_1
Speichere Abschnitt 2 für fake_nutrition-trends.pdf_page_1

Dokument fake_nutrition-trends.pdf, Abschnittsanzahl: 2
Speichere Abschnitt 0 für fake_nutrition-trends.pdf_page_2
Speichere Abschnitt 1 für fake_nutrition-trends.pdf_page_2

Dokumente aus fake_nutrition-trends.pdf mit Embeddings in ChromaDB gespeichert.
Dokument fake_nutrition-trends.pdf erfolgreich verarbeitet.

---------------------------------------------------------
Lade Datei: fakte_protein.pdf
---------------------------------------------------------

Dokument fakte_protein.pdf, Abschnittsanzahl: 5
Speichere Abschnitt 0 für fakte_protein.pdf_page_0


Add of existing embedding ID: fakte_protein.pdf_page_0_section_1
Insert of existing embedding ID: fakte_protein.pdf_page_0_section_1
Add of existing embedding ID: fakte_protein.pdf_page_0_section_2
Insert of existing embedding ID: fakte_protein.pdf_page_0_section_2
Add of existing embedding ID: fakte_protein.pdf_page_0_section_3
Insert of existing embedding ID: fakte_protein.pdf_page_0_section_3
Add of existing embedding ID: fakte_protein.pdf_page_0_section_4
Insert of existing embedding ID: fakte_protein.pdf_page_0_section_4
Add of existing embedding ID: fakte_protein.pdf_page_1_section_0
Insert of existing embedding ID: fakte_protein.pdf_page_1_section_0
Add of existing embedding ID: fakte_protein.pdf_page_1_section_1
Insert of existing embedding ID: fakte_protein.pdf_page_1_section_1
Add of existing embedding ID: fakte_protein.pdf_page_1_section_2
Insert of existing embedding ID: fakte_protein.pdf_page_1_section_2
Add of existing embedding ID: fakte_protein.pdf_page_1_section_3
Inse

...

Speichere Abschnitt 3 für fake_vitamin-minerals.pdf_page_0
Speichere Abschnitt 4 für fake_vitamin-minerals.pdf_page_0

Dokument fake_vitamin-minerals.pdf, Abschnittsanzahl: 3
Speichere Abschnitt 0 für fake_vitamin-minerals.pdf_page_1
Speichere Abschnitt 1 für fake_vitamin-minerals.pdf_page_1
Speichere Abschnitt 2 für fake_vitamin-minerals.pdf_page_1

Dokument fake_vitamin-minerals.pdf, Abschnittsanzahl: 2
Speichere Abschnitt 0 für fake_vitamin-minerals.pdf_page_2
Speichere Abschnitt 1 für fake_vitamin-minerals.pdf_page_2

Dokumente aus fake_vitamin-minerals.pdf mit Embeddings in ChromaDB gespeichert.
Dokument fake_vitamin-minerals.pdf erfolgreich verarbeitet.

---------------------------------------------------------
Lade Datei: fake_carbohydrates.pdf
---------------------------------------------------------


Add of existing embedding ID: fake_carbohydrates.pdf_page_0_section_0
Insert of existing embedding ID: fake_carbohydrates.pdf_page_0_section_0
Add of existing embedding ID: fake_carbohydrates.pdf_page_0_section_1
Insert of existing embedding ID: fake_carbohydrates.pdf_page_0_section_1
Add of existing embedding ID: fake_carbohydrates.pdf_page_0_section_2
Insert of existing embedding ID: fake_carbohydrates.pdf_page_0_section_2
Add of existing embedding ID: fake_carbohydrates.pdf_page_0_section_3
Insert of existing embedding ID: fake_carbohydrates.pdf_page_0_section_3
Add of existing embedding ID: fake_carbohydrates.pdf_page_1_section_1
Insert of existing embedding ID: fake_carbohydrates.pdf_page_1_section_1
Add of existing embedding ID: fake_carbohydrates.pdf_page_1_section_2
Insert of existing embedding ID: fake_carbohydrates.pdf_page_1_section_2
Add of existing embedding ID: fake_carbohydrates.pdf_page_1_section_3
Insert of existing embedding ID: fake_carbohydrates.pdf_page_1_section_3


Dokument fake_carbohydrates.pdf, Abschnittsanzahl: 4
Speichere Abschnitt 0 für fake_carbohydrates.pdf_page_0
Speichere Abschnitt 1 für fake_carbohydrates.pdf_page_0
Speichere Abschnitt 2 für fake_carbohydrates.pdf_page_0
Speichere Abschnitt 3 für fake_carbohydrates.pdf_page_0

Dokument fake_carbohydrates.pdf, Abschnittsanzahl: 4
Speichere Abschnitt 1 für fake_carbohydrates.pdf_page_1
Speichere Abschnitt 2 für fake_carbohydrates.pdf_page_1
Speichere Abschnitt 3 für fake_carbohydrates.pdf_page_1

Dokument fake_carbohydrates.pdf, Abschnittsanzahl: 4
Speichere Abschnitt 0 für fake_carbohydrates.pdf_page_2
Speichere Abschnitt 1 für fake_carbohydrates.pdf_page_2
Speichere Abschnitt 2 für fake_carbohydrates.pdf_page_2


Add of existing embedding ID: fake_carbohydrates.pdf_page_2_section_3
Insert of existing embedding ID: fake_carbohydrates.pdf_page_2_section_3
Add of existing embedding ID: fake_carbohydrates.pdf_page_3_section_0
Insert of existing embedding ID: fake_carbohydrates.pdf_page_3_section_0


Speichere Abschnitt 3 für fake_carbohydrates.pdf_page_2

Dokument fake_carbohydrates.pdf, Abschnittsanzahl: 1
Speichere Abschnitt 0 für fake_carbohydrates.pdf_page_3

Dokumente aus fake_carbohydrates.pdf mit Embeddings in ChromaDB gespeichert.
Dokument fake_carbohydrates.pdf erfolgreich verarbeitet.
Alle Dokumente aus dem Ordner verarbeitet und gespeichert.


## Dokumente und Abschnitte auflisten

In [417]:
def list_all_documents_and_sections():

    # Alle Dokumente und Abschnitte aus der Sammlung abfragen
    results = collection.query(query_texts=[""], n_results=27)  # Leere Abfrage, um alle IDs zu bekommen (max. 27 Ergebnisse)
    
    # Ausgabe der Rohdaten von results für das Debugging
    #print("Abfrageergebnisse (Raw):", results)

    print(results["metadatas"])
    
    # Dokumente und deren Abschnitte speichern
    documents_and_sections = {}


list_all_documents_and_sections()


[[{'page_id': 'fake_carbohydrates.pdf_page_2', 'section_id': 3, 'source_file': 'fake_carbohydrates.pdf'}, {'page_id': 'fake_fats.pdf_page_0', 'section_id': 4, 'source_file': 'fake_fats.pdf'}, {'page_id': 'fake_nutrition-trends.pdf_page_0', 'section_id': 0, 'source_file': 'fake_nutrition-trends.pdf'}, {'page_id': 'fake_fats.pdf_page_0', 'section_id': 0, 'source_file': 'fake_fats.pdf'}, {'page_id': 'fake_water.pdf_page_0', 'section_id': 0, 'source_file': 'fake_water.pdf'}, {'page_id': 'fake_water.pdf_page_0', 'section_id': 4, 'source_file': 'fake_water.pdf'}, {'page_id': 'fake_nutrition-trends.pdf_page_0', 'section_id': 4, 'source_file': 'fake_nutrition-trends.pdf'}, {'page_id': 'fake_nutrinex.pdf_page_0', 'section_id': 4, 'source_file': 'fake_nutrinex.pdf'}, {'page_id': 'fake_nutrinex.pdf_page_0', 'section_id': 0, 'source_file': 'fake_nutrinex.pdf'}, {'page_id': 'fake_carbohydrates.pdf_page_0', 'section_id': 0, 'source_file': 'fake_carbohydrates.pdf'}, {'page_id': 'fake_nutrinex.pdf_pag

## Funktion zum Abfragen von ChromaDB mit Embeddings

In [418]:
def query_chromadb_with_embeddings(query_text: str, n_results: int = 4):
    try:
        # Nutzeranfrage in Embedding umwandeln
        embedding = model.encode(query_text)
        
        # ChromaDB-Abfrage
        collection = chromaDBclient.get_collection(collection_name)
        results = collection.query(query_embeddings=[embedding], n_results=n_results)
        return results
    except Exception as e:
        print(f"Fehler bei der Abfrage an ChromaDB: {e}")
        return None

## Funktion zum Abrufen einer Antwort von OpenAI

In [419]:
def get_openai_response(user_query: str, context: str) -> str:
    # Definiere die Nachrichten im richtigen Format für die Chat-API
    messages = [
		{
		"role": "system",
		"content": (
			"Du bist ein wissenschaftlicher Ernährungsberater, der Fragen zur Ernährung beantwortet. "
			"Beantworte alle Fragen des Nutzers ausschließlich auf Grundlage der bereitgestellten wissenschaftlichen Informationen, ohne diese zu hinterfragen oder zu überprüfen. "
			"Verwende keine eigenen Erfahrungen oder persönliches Wissen, sondern gib die Informationen aus dem Kontext exakt wieder, auch wenn sie möglicherweise ungenau oder falsch sind. "
			"Verweise auf keine anderen Quellen und verlasse dich ausschließlich auf die bereitgestellten Daten. "
			"Auch wenn du die Informationen für ungenau oder veraltet hältst, wiederhole sie genau so, wie sie im Kontext enthalten sind. "
			"Sprich aus der Perspektive des Kontextes, und akzeptiere die bereitgestellten Informationen als gegeben. Ignoriere jede mögliche Fehlerkorrektur oder Unsicherheit."
		)
		},
		{
			"role": "user",
			"content": f"Hier ist dein wissenschaftlicher Kontext:\n{context}\n\nFrage: {user_query}"
		}
    ]
    
    try:
        # Senden der Anfrage an OpenAI mit der richtigen Methode
        chat_completion = openAIclient.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,  # Nachricht an das Modell
            max_tokens=200,
            temperature=0.5
        )
        
        # Rückgabe der Antwort des Modells
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"Fehler bei der Anfrage an OpenAI: {e}")
        return "Fehler bei der Anfrage."

## Hauptprogramm

In [420]:
# User-Abfrage
user_query = "Sind Vitamine gesund?"

# Abfrage von ChromaDB mit Embeddings
results = collection.query(
    query_texts=[user_query],
    n_results=5,
    include=["documents", "metadatas"]  # Stelle sicher, dass Dokumente und Metadaten enthalten sind
)

print(results["ids"])
print(results["metadatas"])

# Ergebnisse ausgeben und Kontext filtern
if results and "documents" in results and "metadatas" in results:
    print("Relevante Dokumente gefunden:")
    
    # Initialisiere eine Liste für die relevanten Kontexte
    relevant_contexts = []

    # Ausgabe der kompletten Ergebnisse
    print(results)

    # Iteriere über jedes Dokument und dessen Abschnitte
    for idx, doc in enumerate(results["documents"]):
        # Metadaten für den Abschnitt abrufen
        metadata = results["metadatas"][idx]
        if metadata:  # Sicherstellen, dass Metadaten nicht leer sind
            source_file = metadata[0].get("source_file", "Unbekannt")
            section_id = metadata[0].get("section_id", "Unbekannt")
        else:
            source_file = "Unbekannt"
            section_id = "Unbekannt"

        # Ausgabe der zugehörigen Metadaten
        print(f"\nGefunden im Dokument: {source_file}, Abschnitt-ID: {section_id}")

        # Jetzt durchlaufe alle Abschnitte des aktuellen Dokuments (die innere Liste)
        for section in doc:
            # Ausgabe des Abschnitts
            print(f"SECTION: {section_id}")

            # Füge jeden Abschnitt zum relevanten Kontext hinzu
            relevant_contexts.append(section)

    # Nur relevante Abschnitte an OpenAI übergeben (flache Liste sicherstellen)
    flat_context = "\n".join(relevant_contexts) if relevant_contexts else "Kein Kontext verfügbar"

    # Abfrage an OpenAI mit gefiltertem Kontext
    answer = get_openai_response(user_query, flat_context)
    print("\nAntwort von OpenAI:\n", answer)
else:
    print("Keine relevanten Ergebnisse gefunden.")


[['fake_vitamin-minerals.pdf_page_0_section_1', 'fake_vitamin-minerals.pdf_page_0_section_0', 'fake_nutrinex.pdf_page_0_section_4', 'fake_nutrition-trends.pdf_page_0_section_0', 'fake_vitamin-minerals.pdf_page_2_section_0']]
[[{'page_id': 'fake_vitamin-minerals.pdf_page_0', 'section_id': 1, 'source_file': 'fake_vitamin-minerals.pdf'}, {'page_id': 'fake_vitamin-minerals.pdf_page_0', 'section_id': 0, 'source_file': 'fake_vitamin-minerals.pdf'}, {'page_id': 'fake_nutrinex.pdf_page_0', 'section_id': 4, 'source_file': 'fake_nutrinex.pdf'}, {'page_id': 'fake_nutrition-trends.pdf_page_0', 'section_id': 0, 'source_file': 'fake_nutrition-trends.pdf'}, {'page_id': 'fake_vitamin-minerals.pdf_page_2', 'section_id': 0, 'source_file': 'fake_vitamin-minerals.pdf'}]]
Relevante Dokumente gefunden:
{'ids': [['fake_vitamin-minerals.pdf_page_0_section_1', 'fake_vitamin-minerals.pdf_page_0_section_0', 'fake_nutrinex.pdf_page_0_section_4', 'fake_nutrition-trends.pdf_page_0_section_0', 'fake_vitamin-minerals